## Merge datasets

In [ ]:
train_df = train_transaction.merge(train_identity, how="left", on="TransactionID")

test_df = test_transaction.merge(test_identity, how="left", on="TransactionID")

## Outliers


https://www.kaggle.com/code/omerparlak/1-fraud-detection-eda#Outliers

In [ ]:
def outlier_thresholds(dataframe, col_name, q1=0.01, q3=0.99):
    quartile1 = dataframe[col_name].quantile(q1)
    quartile3 = dataframe[col_name].quantile(q3)
    interquantile_range = quartile3 - quartile1
    up_limit = quartile3 + 1.5 * interquantile_range
    low_limit = quartile1 - 1.5 * interquantile_range
    return low_limit, up_limit

In [ ]:
def check_outlier(dataframe, col_name):
    low_limit, up_limit = outlier_thresholds(dataframe, col_name)
    if dataframe[(dataframe[col_name] > up_limit) | (dataframe[col_name] < low_limit)].any(axis=None):
        return True
    else:
        return False

In [ ]:
for col in num_cols:
    print(col, check_outlier(train_df, col))

## Correlation Anaylsis


In [ ]:
def high_correlated_cols(dataframe, plot=True, corr_th=0.85):
    corr = dataframe.corr()
    cor_matrix = corr.abs()
    upper_triangle_matrix = cor_matrix.where(np.triu(np.ones(cor_matrix.shape), k=1).astype(np.bool))
    drop_list = [col for col in upper_triangle_matrix.columns if any(upper_triangle_matrix[col] > corr_th)]
    if plot:
        
        sns.set(rc={'figure.figsize': (25,25)})
        sns.heatmap(corr, cmap="RdBu_r", annot=True, fmt='.2f', square=True, linewidths=1,vmin = -1 , vmax = 1)
        plt.show()
    return drop_list

In [ ]:
cols = ['V'+str(x) for x in range(1,26)]
cols = cols + ["isFraud"]
high_correlated_cols(train_df[cols], plot=True)
drop_list1 = high_correlated_cols(train_df[cols], plot=False)

In [ ]:
cols = ['V'+str(x) for x in range(26,51)]
cols = cols + ["isFraud"]
high_correlated_cols(train_df[cols], plot=True)
drop_list2 = high_correlated_cols(train_df[cols], plot=False)

## V Variables with Low Effect by Feature Importance

代码来源： https://www.kaggle.com/code/omerparlak/2-correlation-analysis-and-feature-importance#V-Variables-with-Low-Effect-by-Feature-Importance 

In [ ]:
vfeatures_importance = []
for i in train_df.columns:
    if 'V' in i:
        vfeatures_importance.append(i)

In [ ]:
vfeatures_importance.append('isFraud')

In [ ]:
len(vfeatures_importance)

In [ ]:
v_train = train_df[:412785]
v_cv = train_df[412785:]

In [ ]:
v_train = v_train[vfeatures_importance]
v_cv = v_cv[vfeatures_importance]

In [ ]:
v_train_x  = v_train.drop(['isFraud'],axis=1)
v_train_y = v_train['isFraud']
v_cv_x = v_cv.drop(['isFraud'],axis=1)
v_cv_y = v_cv['isFraud']

In [ ]:
v_train_x.fillna(v_train_x.mean(),inplace=True)
v_cv_x.fillna(v_cv_x.mean(),inplace=True)

In [ ]:
rf = LGBMClassifier()
rf.fit(v_train_x,v_train_y)
predict_y_=rf.predict_proba(v_train_x)
predict_y = rf.predict_proba(v_cv_x)
print('train auc:',roc_auc_score(v_train_y,predict_y_[:,1]))
print('cv auc:',roc_auc_score(v_cv_y,predict_y[:,1]))

In [ ]:
g = rf.feature_importances_
j=1
vremove=[]
for i in g:
    if i<10:
        vremove.append('V'+str(j))
    j+=1

In [ ]:
for i in vremove:
    vfeatures_importance.remove(i)
    print('removed:',i)

In [ ]:
drop_list_vfeatures = [col for col in vremove if col in  drop_list_vfeatures] 

*** 以上代码来源： https://www.kaggle.com/code/omerparlak/2-correlation-analysis-and-feature-importance#V-Variables-with-Low-Effect-by-Feature-Importance   ***